# CH-05 AI 技術指標回測系統

## 5-2 強大的回測工具：backtesting.py

### 1️⃣ 安裝及匯入套件

In [1]:
!pip install openai
!pip install yfinance
!pip install backtesting
!pip install bokeh # 繪圖套件

In [2]:
from  openai import OpenAI, OpenAIError # 串接 OpenAI API
import yfinance as yf
import pandas as pd # 資料處理套件
import datetime as dt # 時間套件
from backtesting import Backtest, Strategy
from backtesting.lib import crossover
from backtesting.test import SMA

/usr/local/lib/python3.12/dist-packages/backtesting/_plotting.py:55: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support, such as old IDEs. Reset with `backtesting.set_bokeh_output(notebook=False)`.
  warnings.warn('Jupyter Notebook detected. '


### 2️⃣ 取得股價資料

In [3]:
# 輸入股票代號
stock_id = "2330.tw"
# 抓取 5 年資料
#df = yf.download(stock_id, period="5y")
df = yf.Ticker(stock_id).history(period="5y")
# 計算指標
df['ma1'] = df['Close'].rolling(window=5).mean()
df['ma2'] = df['Close'].rolling(window=10).mean()
df.head()

,Open,High,Low,Close,Volume,Dividends,Stock Splits,ma1,ma2
Date,,,,,,,,,
2020-10-19 00:00:00+08:00,412.103462,416.197205,410.284020,416.197205,34470906,0.0,0.0,NaN,NaN
2020-10-20 00:00:00+08:00,414.377709,415.742289,410.283966,410.283966,25205638,0.0,0.0,NaN,NaN
2020-10-21 00:00:00+08:00,412.558284,414.832586,410.283983,412.103424,28058921,0.0,0.0,NaN,NaN
2020-10-22 00:00:00+08:00,409.374219,413.922821,408.919358,413.922821,23438547,0.0,0.0,NaN,NaN
2020-10-23 00:00:00+08:00,416.651988,416.651988,410.738804,411.193665,18777385,0.0,0.0,412.740216,NaN


### 3️⃣ 定義回測策略

In [4]:
class CrossStrategy(Strategy):
  def init(self):
    super().init()

  def next(self):
    if crossover(self.data.ma1, self.data.ma2):
      self.buy(size=1)
    elif crossover(self.data.ma2, self.data.ma1):
      self.sell(size=1)

### 4️⃣ 回測結果

In [5]:
backtest = Backtest(df,
        CrossStrategy,
        cash=100000,
        commission=0.004,
        margin=1,
        hedging=False,
        trade_on_close=False,
        exclusive_orders=False,
        )
stats = backtest.run()

# 印出回測績效
print(stats)

# 查看詳細的交易紀錄
stats["_trades"].head()

Backtest.run:   0%|          | 0/1214 [00:00<?, ?bar/s]

Start                     2020-10-19 00:00...
End                       2025-10-17 00:00...
Duration                   1824 days 00:00:00
Exposure Time [%]                    70.45267
Equity Final [$]                 101782.80217
Equity Peak [$]                  101887.80217
Commissions [$]                     314.26734
Return [%]                             1.7828
Buy & Hold Return [%]               248.39254
Return (Ann.) [%]                     0.36718
Volatility (Ann.) [%]                  0.4224
CAGR [%]                              0.24444
Sharpe Ratio                          0.86928
Sortino Ratio                         1.33161
Calmar Ratio                          0.44446
Alpha [%]                            -0.29311
Beta                                  0.00836
Max. Drawdown [%]                    -0.82613
Avg. Drawdown [%]                    -0.10477
Max. Drawdown Duration     1120 days 00:00:00
Avg. Drawdown Duration       68 days 00:00:00
# Trades                          

/tmp/ipython-input-4280171815.py:10: UserWarning: Some trades remain open at the end of backtest. Use `Backtest(..., finalize_trades=True)` to close them and include them in stats.
  stats = backtest.run()


,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,SL,TP,PnL,Commission,ReturnPct,EntryTime,ExitTime,Duration,Tag
0,1,15,31,416.652043,445.308171,None,None,25.208287,3.447841,0.060502,2020-11-09 00:00:00+08:00,2020-12-01 00:00:00+08:00,22 days,None
1,1,34,43,453.495704,470.804961,None,None,13.612054,3.697203,0.030016,2020-12-04 00:00:00+08:00,2020-12-17 00:00:00+08:00,13 days,None
2,1,47,73,464.405640,565.880185,None,None,97.353402,4.121143,0.209630,2020-12-23 00:00:00+08:00,2021-01-29 00:00:00+08:00,37 days,None
3,1,79,86,606.104126,558.566526,None,None,-52.196282,4.658683,-0.086118,2021-02-17 00:00:00+08:00,2021-02-26 00:00:00+08:00,9 days,None
4,1,98,102,561.771890,548.003052,None,None,-18.207938,4.439100,-0.032412,2021-03-17 00:00:00+08:00,2021-03-23 00:00:00+08:00,6 days,None


### 5️⃣ 回測繪圖

In [6]:
backtest.plot(plot_equity=True,
       plot_return=False,
       plot_pl=True,
       plot_volume=True,
       plot_drawdown=False,
       superimpose=True)

/usr/local/lib/python3.12/dist-packages/bokeh/util/serialization.py:242: UserWarning: no explicit representation of timezones available for np.datetime64
  return convert(array.astype("datetime64[us]"))


GridPlot(id='p1329', ...)

### 6️⃣ 設定停利、停損點

In [7]:
class CrossStrategy(Strategy):
  def init(self):
    super().init()

  def next(self):
    if crossover(self.data.ma1, self.data.ma2):
        # 買入時設置停損與停利價格
        self.buy(size=1,
            sl=self.data.Close[-1] * 0.90,
            tp=self.data.Close[-1] * 1.10)
    elif crossover(self.data.ma2, self.data.ma1):
        # 賣出時時設置停損與停利價格
        self.sell(size=1,
             sl=self.data.Close[-1] * 1.10,
             tp=self.data.Close[-1] * 0.90)

backtest = Backtest(df,
        CrossStrategy,
        cash=100000,
        commission=0.004,
        margin=1,
        hedging=False,
        trade_on_close=False,
        exclusive_orders=False,
        )
stats = backtest.run()
print(stats)

Backtest.run:   0%|          | 0/1214 [00:00<?, ?bar/s]

Start                     2020-10-19 00:00...
End                       2025-10-17 00:00...
Duration                   1824 days 00:00:00
Exposure Time [%]                    48.64198
Equity Final [$]                  99634.54491
Equity Peak [$]                  100050.74374
Commissions [$]                     355.88927
Return [%]                           -0.36546
Buy & Hold Return [%]               248.39254
Return (Ann.) [%]                    -0.07591
Volatility (Ann.) [%]                 0.15433
CAGR [%]                             -0.05057
Sharpe Ratio                         -0.49185
Sortino Ratio                        -0.67073
Calmar Ratio                         -0.18248
Alpha [%]                            -0.55851
Beta                                  0.00078
Max. Drawdown [%]                    -0.41599
Avg. Drawdown [%]                     -0.1462
Max. Drawdown Duration     1722 days 00:00:00
Avg. Drawdown Duration      597 days 00:00:00
# Trades                          

## 5-3 讓 AI 產生回測策略

### 7️⃣ 輸入 OpenAI API KEY

In [8]:
from google.colab import userdata

client = OpenAI(
    api_key=userdata.get('GEMINI_API_KEY'),
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/"
)

### 8️⃣ 創建 Gemini-2.5-Flash 模型函式

In [9]:
# Gemini 2.5 Flash 模型
def get_reply(messages):
  try:
    response = client.chat.completions.create(
        model="gemini-2.5-flash",
        n=1,
        messages=messages)
    reply = response.choices[0].message.content
  except OpenAIError as err:
    reply = f"發生 {err.type} 錯誤\n{err.message}"
  return reply

# 設定 AI 角色, 使其依據使用者需求進行 df 處理
def ai_helper(df, user_msg):

  msg = [{
    "role":
    "system",
    "content":
    f"As a professional code generation robot, \
      I require your assistance in generating Python code \
      based on specific user requirements. To proceed, \
      I will provide you with a dataframe (df) that follows the \
      format {df.columns}. Your task is to carefully analyze the \
      user's requirements and generate the Python code \
      accordingly.Please note that your response should solely \
      consist of the code itself, \
      and no additional information should be included."
  }, {
    "role":
    "user",
    "content":
    f"The user requirement:{user_msg} \n\
      Your task is to develop a Python function named \
      'calculate(df)'. This function should accept a dataframe as \
      its parameter. Ensure that you only utilize the columns \
      present in the dataset, specifically {df.columns}.\
      After processing, the function should return the processed \
      dataframe. Your response should strictly contain the Python \
      code for the 'calculate(df)' function \
      and exclude any unrelated content, do not import backtrader"
  }]

  reply_data = get_reply(msg)
  return reply_data

# 產生技術指標策略
def ai_strategy(df, user_msg, add_msg="無"):

  code_example ='''
class AiStrategy(Strategy):
  def init(self):
    super().init()

  def next(self):
    if crossover(self.data.short_ma, self.data.long_ma):
        self.buy(size=1,
            sl=self.data.Close[-1] * 0.90,
            tp=self.data.Close[-1] * 1.10)
    elif crossover(self.data.long_ma, self.data.short_ma):
        self.sell(size=1,
             sl=self.data.Close[-1] * 1.10,
             tp=self.data.Close[-1] * 0.90)
        '''

  msg = [{
    "role":
    "system",
    "content":
     "As a Python code generation bot, your task is to generate \
     code for a stock strategy based on user requirements and df. \
     Please note that your response should solely \
     consist of the code itself, \
     and no additional information should be included."
  }, {
    "role":
    "user",
    "content":
     "The user requirement:計算 ma,\n\
     The additional requirement: 請設置 10% 的停利與停損點\n\
     The df.columns =['Open',	'High', 'Low',	'Close',	'Adj Close',	'Volume', 'short_ma',	'long_ma']\n\
     Please using the crossover() function in next(self)\
     Your response should strictly contain the Python \
     code for the 'AiStrategy(Strategy)' class \
     and exclude any unrelated content, don not import backtrader."
  }, {
    "role":
    "assistant",
    "content":f"{code_example}"
  }, {
    "role":
    "user",
    "content":
    f"The user requirement:{user_msg}\n\
     The additional requirement:{add_msg}\n\
     The df.columns ={df.columns}\n\
     Your task is to develop a Python class named \
     'AiStrategy(Strategy)'\
     Please using the crossover() function in next(self), do not import backtrader"

  }]

  reply_data = get_reply(msg)
  return reply_data


### 9️⃣ 計算技術指標

In [10]:
# 輸入股票代號
stock_id = "2330.tw"
# 抓取 5 年資料
#df = yf.download(stock_id, period="5y")
df = yf.Ticker(stock_id).history(period="5y")
# 計算指標
user_msg = ["MACD", "請設置10%的停損點與20%的停利點"]
#user_msg = ["RSI", "請設置10%的停損點與20%的停利點"]
code_str = ai_helper(df, user_msg[0])
code_str=code_str.replace('```','')
code_str=code_str.replace('python','')
print(code_str)
exec(code_str)
new_df = calculate(df)
new_df.tail()


import pandas as pd

def calculate(df):
    """
    Calculates the Moving Average Convergence Divergence (MACD) indicator for the given dataframe.

    Args:
        df (pd.DataFrame): The input dataframe containing 'Close' prices.

    Returns:
        pd.DataFrame: The dataframe with MACD, Signal Line, and MACD Histogram columns added.
    """
    # Calculate the 12-period Exponential Moving Average (EMA)
    df['EMA_12'] = df['Close'].ewm(span=12, adjust=False).mean()

    # Calculate the 26-period Exponential Moving Average (EMA)
    df['EMA_26'] = df['Close'].ewm(span=26, adjust=False).mean()

    # Calculate the MACD Line
    df['MACD'] = df['EMA_12'] - df['EMA_26']

    # Calculate the Signal Line (9-period EMA of the MACD Line)
    df['MACD_Signal'] = df['MACD'].ewm(span=9, adjust=False).mean()

    # Calculate the MACD Histogram
    df['MACD_Hist'] = df['MACD'] - df['MACD_Signal']

    # Drop the intermediate EMA columns if not needed in the final output
    df = df.drop(colu

,Open,High,Low,Close,Volume,Dividends,Stock Splits,MACD,MACD_Signal,MACD_Hist
Date,,,,,,,,,,
2025-10-13 00:00:00+08:00,1390.0,1420.0,1390.0,1415.0,53850992,0.0,0.0,58.932698,50.599089,8.333609
2025-10-14 00:00:00+08:00,1455.0,1460.0,1420.0,1425.0,39889256,0.0,0.0,58.900610,52.259393,6.641217
2025-10-15 00:00:00+08:00,1435.0,1465.0,1425.0,1465.0,41056405,0.0,0.0,61.395122,54.086539,7.308583
2025-10-16 00:00:00+08:00,1465.0,1495.0,1465.0,1485.0,37737678,0.0,0.0,64.245293,56.118290,8.127003
2025-10-17 00:00:00+08:00,1455.0,1465.0,1450.0,1450.0,38077331,0.0,0.0,62.954170,57.485466,5.468704


### 🔟 策略生成

In [11]:
strategy_str = ai_strategy(new_df, user_msg[0], user_msg[1])
strategy_str=strategy_str.replace('```','')
strategy_str=strategy_str.replace('python','')
print(strategy_str)
print("-----------------------")
exec(strategy_str)
backtest = Backtest(df,
        AiStrategy,
        cash=100000,
        commission=0.004,
        trade_on_close=True,
        exclusive_orders=True,
        )
stats = backtest.run()
print(stats)

class AiStrategy(Strategy):
  def init(self):
    super().init()

  def next(self):
    if crossover(self.data.MACD, self.data.MACD_Signal):
        self.buy(size=1,
            sl=self.data.Close[-1] * 0.90,
            tp=self.data.Close[-1] * 1.20)
    elif crossover(self.data.MACD_Signal, self.data.MACD):
        self.sell(size=1,
             sl=self.data.Close[-1] * 1.10,
             tp=self.data.Close[-1] * 0.80)
-----------------------


Backtest.run:   0%|          | 0/1214 [00:00<?, ?bar/s]

Start                     2020-10-19 00:00...
End                       2025-10-17 00:00...
Duration                   1824 days 00:00:00
Exposure Time [%]                    95.72016
Equity Final [$]                  99872.56968
Equity Peak [$]                  100287.24697
Commissions [$]                     550.31418
Return [%]                           -0.12743
Buy & Hold Return [%]               248.39254
Return (Ann.) [%]                    -0.02644
Volatility (Ann.) [%]                 0.21073
CAGR [%]                             -0.01762
Sharpe Ratio                         -0.12548
Sortino Ratio                        -0.17804
Calmar Ratio                         -0.04154
Alpha [%]                              0.1591
Beta                                 -0.00115
Max. Drawdown [%]                    -0.63659
Avg. Drawdown [%]                    -0.07782
Max. Drawdown Duration      731 days 00:00:00
Avg. Drawdown Duration       93 days 00:00:00
# Trades                          

### 1️⃣1️⃣ 寫成函式

In [12]:
def ai_backtest(stock_id, period, user_msg, add_msg):

  # 下載資料
  #df = yf.download(stock_id, period=period)
  df = yf.Ticker(stock_id).history(period="5y")

  # 獲取和執行指標計算程式碼
  code_str = ai_helper(df, user_msg)
  code_str=code_str.replace('```','')
  code_str=code_str.replace('python','')
  local_namespace = {}
  exec(code_str, globals(), local_namespace)
  calculate = local_namespace['calculate']
  new_df = calculate(df)

  # 獲取和執行策略程式碼
  strategy_str = ai_strategy(new_df, user_msg, add_msg)
  strategy_str=strategy_str.replace('```','')
  strategy_str=strategy_str.replace('python','')
  print(strategy_str)

  print("-----------------------")
  exec(strategy_str, globals(), local_namespace)
  AiStrategy = local_namespace['AiStrategy']

  backtest = Backtest(df,
          AiStrategy,
          cash=100000,
          commission=0.004,
          trade_on_close=True,
          exclusive_orders=True,
          )
  stats = backtest.run()
  print(stats)
  return str(stats)


## 5-4 讓 AI 解析回測報告

### 1️⃣2️⃣ 設定 AI 回復內容

In [13]:
def backtest_analysis(*args):

  content_list = [f"策略{i+1}：{report}"
                  for i, report in enumerate(args)]
  content = "\n".join(content_list)
  content += "\n\n請依資料給我一份約200字的分析報告。若有多個策略, \
                  請選出最好的策略及原因, reply in 繁體中文."

  msg = [{
      "role": "system",
      "content": "你是一位專業的證券分析師, 我會給你交易策略的回測績效,\
                  請幫我進行績效分析.不用詳細講解每個欄位, \
                  重點說明即可, 並回答交易策略的好壞"
  }, {
      "role": "user",
      "content": content
  }]

  reply_data = get_reply(msg)
  return reply_data


### 1️⃣3️⃣ 回測結果分析

In [14]:
stats = ai_backtest(stock_id="2330.TW",
           period="5y",
           user_msg="MACD",
           add_msg="請設置10%的停損點與20%的停利點")
reply = backtest_analysis(stats)
print(reply)


class AiStrategy(Strategy):
  def init(self):
    super().init()

  def next(self):
    if crossover(self.data.MACD, self.data.Signal_Line):
        self.buy(size=1,
            sl=self.data.Close[-1] * 0.90,  # 10% stop-loss
            tp=self.data.Close[-1] * 1.20)  # 20% take-profit
    elif crossover(self.data.Signal_Line, self.data.MACD):
        self.sell(size=1,
             sl=self.data.Close[-1] * 1.10,  # 10% stop-loss for short position
             tp=self.data.Close[-1] * 0.80)  # 20% take-profit for short position
-----------------------


Backtest.run:   0%|          | 0/1214 [00:00<?, ?bar/s]

Start                     2020-10-19 00:00...
End                       2025-10-17 00:00...
Duration                   1824 days 00:00:00
Exposure Time [%]                    95.72016
Equity Final [$]                  99872.56968
Equity Peak [$]                  100287.24697
Commissions [$]                     550.31418
Return [%]                           -0.12743
Buy & Hold Return [%]               248.39254
Return (Ann.) [%]                    -0.02644
Volatility (Ann.) [%]                 0.21073
CAGR [%]                             -0.01762
Sharpe Ratio                         -0.12548
Sortino Ratio                        -0.17804
Calmar Ratio                         -0.04154
Alpha [%]                              0.1591
Beta                                 -0.00115
Max. Drawdown [%]                    -0.63659
Avg. Drawdown [%]                    -0.07782
Max. Drawdown Duration      731 days 00:00:00
Avg. Drawdown Duration       93 days 00:00:00
# Trades                          

### 1️⃣4️⃣ 比較多個策略

In [16]:
# 策略1:MACD+停利停損
stats1 = ai_backtest(stock_id="2330.TW", period="5y",
            user_msg="MACD",
            add_msg="請設置10%的停損點與20%的停利點")

class AiStrategy(Strategy):
  def init(self):
    super().init()

  def next(self):
    if crossover(self.data.MACD, self.data.MACD_Signal):
        self.buy(size=1,
            sl=self.data.Close[-1] * 0.90,
            tp=self.data.Close[-1] * 1.20)
    elif crossover(self.data.MACD_Signal, self.data.MACD):
        self.sell(size=1,
             sl=self.data.Close[-1] * 1.10,
             tp=self.data.Close[-1] * 0.80)
-----------------------


Backtest.run:   0%|          | 0/1214 [00:00<?, ?bar/s]

Start                     2020-10-19 00:00...
End                       2025-10-17 00:00...
Duration                   1824 days 00:00:00
Exposure Time [%]                    95.72016
Equity Final [$]                  99872.57015
Equity Peak [$]                  100287.24805
Commissions [$]                     550.31419
Return [%]                           -0.12743
Buy & Hold Return [%]               248.39251
Return (Ann.) [%]                    -0.02644
Volatility (Ann.) [%]                 0.21073
CAGR [%]                             -0.01762
Sharpe Ratio                         -0.12548
Sortino Ratio                        -0.17804
Calmar Ratio                         -0.04154
Alpha [%]                              0.1591
Beta                                 -0.00115
Max. Drawdown [%]                    -0.63659
Avg. Drawdown [%]                    -0.07782
Max. Drawdown Duration      731 days 00:00:00
Avg. Drawdown Duration       93 days 00:00:00
# Trades                          

In [18]:
# 策略2:SMA
stats2 = ai_backtest(stock_id="2330.TW", period="5y",
            user_msg="SMA",
            add_msg="無")

class AiStrategy(Strategy):
  def init(self):
    super().init()

  def next(self):
    # Buy signal: Close crosses above SMA
    if crossover(self.data.Close, self.data.SMA):
        self.buy()
    # Sell signal: Close crosses below SMA (SMA crosses above Close)
    elif crossover(self.data.SMA, self.data.Close):
        self.sell()
-----------------------


Backtest.run:   0%|          | 0/1214 [00:00<?, ?bar/s]

Start                     2020-10-19 00:00...
End                       2025-10-17 00:00...
Duration                   1824 days 00:00:00
Exposure Time [%]                    91.76955
Equity Final [$]                  65446.70806
Equity Peak [$]                  103528.61375
Commissions [$]                   67918.14623
Return [%]                          -34.55329
Buy & Hold Return [%]               248.39259
Return (Ann.) [%]                    -8.41723
Volatility (Ann.) [%]                25.08804
CAGR [%]                             -5.68876
Sharpe Ratio                         -0.33551
Sortino Ratio                        -0.45423
Calmar Ratio                         -0.15727
Alpha [%]                           -43.27329
Beta                                  0.03511
Max. Drawdown [%]                   -53.52202
Avg. Drawdown [%]                   -28.64194
Max. Drawdown Duration     1703 days 00:00:00
Avg. Drawdown Duration      861 days 00:00:00
# Trades                          

/tmp/ipython-input-897687173.py:33: UserWarning: Some trades remain open at the end of backtest. Use `Backtest(..., finalize_trades=True)` to close them and include them in stats.
  stats = backtest.run()


In [19]:
# 策略3:RSI+停利停損
stats3 = ai_backtest(stock_id="2330.TW", period="5y",
            user_msg="RSI",
            add_msg="請設置10%的停損點與20%的停利點")

class AiStrategy(Strategy):
  def init(self):
    super().init()

  def next(self):
    # Buy when RSI crosses above 30 (oversold)
    if crossover(self.data.RSI, 30):
        self.buy(size=1,
            sl=self.data.Close[-1] * 0.90,  # 10% stop-loss
            tp=self.data.Close[-1] * 1.20)  # 20% take-profit
    # Sell when RSI crosses below 70 (overbought)
    elif crossover(70, self.data.RSI):
        self.sell(size=1,
             sl=self.data.Close[-1] * 1.10,  # 10% stop-loss for short position
             tp=self.data.Close[-1] * 0.80)  # 20% take-profit for short position
-----------------------


Backtest.run:   0%|          | 0/1214 [00:00<?, ?bar/s]

Start                     2020-10-19 00:00...
End                       2025-10-17 00:00...
Duration                   1824 days 00:00:00
Exposure Time [%]                    87.65432
Equity Final [$]                   99032.2146
Equity Peak [$]                  100154.03804
Commissions [$]                     236.75375
Return [%]                           -0.96779
Buy & Hold Return [%]               248.39259
Return (Ann.) [%]                     -0.2015
Volatility (Ann.) [%]                 0.18379
CAGR [%]                             -0.13427
Sharpe Ratio                         -1.09638
Sortino Ratio                        -1.45067
Calmar Ratio                         -0.17445
Alpha [%]                             -0.4815
Beta                                 -0.00196
Max. Drawdown [%]                    -1.15504
Avg. Drawdown [%]                    -0.16635
Max. Drawdown Duration     1319 days 00:00:00
Avg. Drawdown Duration      199 days 00:00:00
# Trades                          

/tmp/ipython-input-897687173.py:33: UserWarning: Some trades remain open at the end of backtest. Use `Backtest(..., finalize_trades=True)` to close them and include them in stats.
  stats = backtest.run()


In [20]:
reply = backtest_analysis(stats1, stats2, stats3)
print(reply)

綜合來看，這三個策略的表現皆不理想，無法為投資者帶來正向收益，且均大幅落後於「買入並持有」(Buy & Hold) 的基準報酬（248.39%）。

**策略1 表現分析：**
儘管總報酬和年化報酬率皆為負值，但其虧損幅度非常小 (Return: -0.127%)。最值得注意的是，**最大回撤 (Max. Drawdown) 僅為 -0.64%**，顯示其風險控制極為嚴格或波動性極低。交易層面來看，其獲利因子 (Profit Factor) 為 1.002，期望值 (Expectancy) 為 0.005%，表示單次交易平均略有盈利，但勝率偏低。然而，面對市場大幅上漲時，卻未能捕捉到任何漲幅，這點極為不足。

**策略2 表現分析：**
這是表現最差的策略。其總報酬為 -34.55%，年化報酬率 -8.42%。最主要的問題在於**巨額的交易佣金 (Commissions: $67,918)** 嚴重侵蝕了所有收益，以及高達 -53.52% 的最大回撤，代表承擔了極高的風險卻帶來巨大虧損。獲利因子遠低於1 (0.797)，期望值為負，交易勝率僅 17.6%，顯示其交易邏輯存在根本性缺陷。

**策略3 表現分析：**
此策略同樣表現不佳，總報酬為 -0.97%，年化報酬率 -0.20%。儘管最大回撤 (-1.15%) 低於策略2，但其**獲利因子僅為 0.472，期望值為 -2.74%**，是三者中單次交易虧損最嚴重的。勝率也僅 20.45%，表明該策略在交易層面上缺乏盈利能力。

**最佳策略選擇：**
在三者皆表現不佳的情況下，**策略1 相對是「最好」的策略。**
主要原因在於：
1.  **虧損幅度最小：** 總報酬和年化報酬率最接近零，未導致顯著虧損。
2.  **風險控制最佳：** 最大回撤極低 (-0.64%)，雖然沒有盈利，但至少保住了本金。
3.  **交易結構相對健康：** 獲利因子大於 1 且期望值為正，顯示其單次交易平均是微利狀態，只是整體效益未能體現或交易頻率不足。

**總結：**
雖然策略1是三者中相對較佳，但**三個策略都屬於不成功的交易策略**，無法在回測期間內產生正向報酬，並且大幅落後於簡單的市場買入持有策略。策略1或許在風險控制上表現穩健，但缺乏獲利能力；策略2和3則無論在報酬、風險或交易效率上都非常差。建議需對所有策略進行徹底的

## 5-5 策略之回測與繪圖

In [21]:
# 60MA 高於季線做多，跌破出場

class OneMA(Strategy):
    n1 = 60  #預設的均線參數

    def init(self): #初始化會用到的參數和指標，告知要如何計算
        self.sma1 = self.I(SMA, self.data.Close, self.n1)

    def next(self): #回測的時候每一根K棒出現什麼狀況要觸發進出場
        #如果收盤價>sma1(也就是60ma)，而且目前沒有多單部位
        if (self.data.Close > self.sma1) and (not self.position.is_long) :
            self.buy()#做多
        #如果收盤價<sma1(也就是60ma)
        elif (self.data.Close < self.sma1):
            self.position.close()#部位出場
                                 #如果要做空就用self.sell()

In [22]:
backtest = Backtest(df,
        OneMA,
        cash=100000,
        commission=0.004,
        margin=1,
        hedging=False,
        trade_on_close=False,
        exclusive_orders=False,
        )
stats = backtest.run()

# 印出回測績效
print(stats)

# 查看詳細的交易紀錄
stats["_trades"].head()

Backtest.run:   0%|          | 0/1155 [00:00<?, ?bar/s]

Start                     2020-10-19 00:00...
End                       2025-10-17 00:00...
Duration                   1824 days 00:00:00
Exposure Time [%]                    56.87243
Equity Final [$]                 130936.44199
Equity Peak [$]                  134086.44199
Commissions [$]                   22062.67622
Return [%]                           30.93644
Buy & Hold Return [%]               171.59478
Return (Ann.) [%]                     5.74972
Volatility (Ann.) [%]                22.93463
CAGR [%]                              3.79414
Sharpe Ratio                           0.2507
Sortino Ratio                         0.40274
Calmar Ratio                          0.11356
Alpha [%]                           -60.72461
Beta                                  0.53417
Max. Drawdown [%]                   -50.63344
Avg. Drawdown [%]                   -14.78829
Max. Drawdown Duration     1264 days 00:00:00
Avg. Drawdown Duration      288 days 00:00:00
# Trades                          

/tmp/ipython-input-2353848551.py:10: UserWarning: Some trades remain open at the end of backtest. Use `Backtest(..., finalize_trades=True)` to close them and include them in stats.
  stats = backtest.run()


,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,SL,TP,PnL,Commission,ReturnPct,EntryTime,ExitTime,Duration,Tag,"Entry_SMA(C,60)","Exit_SMA(C,60)"
0,183,61,104,543.939699,525.054749,None,None,-4238.449826,782.503936,-0.042580,2021-01-13 00:00:00+08:00,2021-03-25 00:00:00+08:00,71 days,None,452.193248,544.410994
1,173,107,108,550.756836,547.085144,None,None,-1394.909294,759.706650,-0.014640,2021-03-30 00:00:00+08:00,2021-03-31 00:00:00+08:00,1 days,None,548.362006,549.571835
2,166,110,114,563.607808,554.428447,None,None,-2266.150112,742.376073,-0.024222,2021-04-07 00:00:00+08:00,2021-04-13 00:00:00+08:00,6 days,None,552.481425,557.615533
3,163,116,118,561.771832,555.346476,None,None,-1775.694102,728.361137,-0.019392,2021-04-15 00:00:00+08:00,2021-04-19 00:00:00+08:00,4 days,None,559.825437,560.937200
4,163,148,153,548.920898,541.577413,None,None,-1907.993077,711.004899,-0.021325,2021-06-01 00:00:00+08:00,2021-06-08 00:00:00+08:00,7 days,None,543.532965,543.337170


In [23]:
backtest.plot(plot_equity=True,
       plot_return=False,
       plot_pl=True,
       plot_volume=True,
       plot_drawdown=False,
       superimpose=True)

/usr/local/lib/python3.12/dist-packages/bokeh/util/serialization.py:242: UserWarning: no explicit representation of timezones available for np.datetime64
  return convert(array.astype("datetime64[us]"))


GridPlot(id='p1693', ...)

## References:

### [Backtesting - 均線突破策略](https://ithelp.ithome.com.tw/articles/10274546)

### [Backtesting - 參數最佳化](https://ithelp.ithome.com.tw/articles/10274547)